In [ ]:
!pip install transformers

In [2]:
import os, copy
import pandas as pd
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
import warnings

warnings.filterwarnings('ignore')

In [ ]:
origin_df = pd.read_csv('C:/Users/Lee_Hyo_Jae/Desktop/new_project/dataset/intent_model_dataset.csv')
df = copy.deepcopy(origin_df)   
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df.drop(df.loc[df['sentence'].str.contains('\*')].index, axis=0, inplace=True)
df = df.loc[df['sentence'].str.len() <= 100]
df.reset_index(inplace=True, drop=True)


df.loc[(df['sentence'].str.contains('응') & df['sentence'].str.contains('좋아')), 'intent'] = '긍정'
df.loc[(df['sentence'].str.contains('응') & df['sentence'].str.contains('맞아')), 'intent'] = '긍정'

df.loc[(df['sentence'].str.contains('아니') & df['sentence'].str.contains('싫어')), 'intent'] = '부정'


fst_P_key = ['맞어','맞아','마자','마쟈','마저','맞지','응 ','응!','응응 ','웅 ','웅웅 ','웅!','좋아 ','좋다']
fst_N_key = ['아뇨','아니','아닌데','아녀','아니여','아냐','싫어 ','싫어,','싫어!','시러','싫은데','안돼','안되지','안된','별로야']

for i in fst_P_key:
    df.loc[(df['sentence'].str.startswith(i)), 'intent'] = '긍정'
for i in fst_N_key:
    df.loc[(df['sentence'].str.startswith(i)), 'intent'] = '부정'

df

In [ ]:
asdf = list(df['intent'].unique())
for i in asdf :
    print(i,df.loc[df['intent']==i,'intent'].count())

In [31]:
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

In [ ]:
# token 바구니
N_count = []

# 필요 문장 추출
N_intent = ['(표현) 부정감정 표현하기', '(단언) 반박하기', '(언약) 거절하기', '부정']

# 필요 문장 구성하는 모든 token 바구니에 넣기
for intent in N_intent :
    for i in df.loc[(df['intent'] == intent), 'sentence']:
        N_count += tokenizer.encode(i)

# token 고유값과 개수 / N_uni[0] = 고유값 리스트 / N_uni[1] = 고유값 개수 리스트
N_uni = np.unique(N_count, return_counts=True)

# 고유값 개수 내림차순 정렬
N_cnt = sorted(list(set(N_uni[1])),reverse=True)

# N_many = N_uni[0]에서 검색할 index값 / N_token = 사용 횟수가 많은 token
N_many = []
N_token = []

for i in N_cnt:
    if i > 2700 :
        N_many.append(list(N_uni[1]).index(i))
for j in range(len(N_many)) :
    N_token.append(N_uni[0][N_many[j]])
print(N_many)

tokenizer.decode(N_token)

In [ ]:
# token 바구니
P_count = []

# 필요 문장 추출
P_intent = ['(표현) 긍정감정 표현하기', '(표현) 감사하기', '긍정']

# 필요 문장 구성하는 모든 token 바구니에 넣기
for intent in P_intent :
    for i in df.loc[(df['intent'] == intent), 'sentence']:
        P_count += tokenizer.encode(i)

# token 고유값과 개수 / N_uni[0] = 고유값 리스트 / N_uni[1] = 고유값 개수 리스트
P_uni = np.unique(P_count, return_counts=True)

# 고유값 개수 내림차순 정렬
P_cnt = sorted(list(set(P_uni[1])),reverse=True)

# N_many = N_uni[0]에서 검색할 index값 / N_token = 사용 횟수가 많은 token
P_many = []
P_token = []

for i in P_cnt:
    if i > 7700 :
        P_many.append(list(P_uni[1]).index(i))
for j in range(len(P_many)) :
    P_token.append(P_uni[0][P_many[j]])
print(P_many)

tokenizer.decode(P_token)

In [ ]:
# token 바구니
M_count = []

# 필요 문장 추출
M_intent = ['(단언) 진술하기', '(지시) 충고/제안하기', '(단언) 주장하기', '(지시) 질문하기', '(지시) 부탁하기',
        '(표현) 사과하기', '(지시) 명령/요구하기', '턴토크 사인(관습적 반응)',
       '(언약) 약속하기(제3자와)/(개인적 수준)', '(표현) 인사하기', '(선언/위임하기)']

# 필요 문장 구성하는 모든 token 바구니에 넣기
for intenti in range(len(M_intent)):
    if df.loc[(df['intent'] == intent), 'sentence'].count() > 1000:
        for i in (df.loc[(df['intent'] == intent), 'sentence']):
            M_count += tokenizer.encode(i)
    else:
        for i in (df.loc[(df['intent'] == intent), 'sentence']):
            M_count += tokenizer.encode(i)

# token 고유값과 개수 / N_uni[0] = 고유값 리스트 / N_uni[1] = 고유값 개수 리스트
M_uni = np.unique(M_count, return_counts=True)

# 고유값 개수 내림차순 정렬
M_cnt = sorted(list(set(M_uni[1])),reverse=True)

# M_many = N_uni[0]에서 검색할 index값 / M_token = 사용 횟수가 많은 token
M_many = []
M_token = []

for i in M_cnt:
    if i > 100000 :
        M_many.append(list(M_uni[1]).index(i))
for j in range(len(M_many)) :
    M_token.append(M_uni[0][M_many[j]])
print(M_many)

tokenizer.decode(M_token)

In [ ]:
# 중복값 제거
print(tokenizer.decode((set(N_token).difference(set(P_token))).difference(set(M_token))))
print(tokenizer.decode((set(P_token).difference(set(N_token))).difference(set(M_token))))

In [38]:
N_keyword = ['싫어\.\.', '아니']
P_keyword = ['맞아', '응응', '응\.', '응,']

In [ ]:
# 부정키워드 포함 긍정문장 제거
for intent in P_intent:
    for key in N_keyword:
        df.drop(df.loc[(df['intent'] ==intent)&df['sentence'].str.contains(key)].index, axis=0, inplace=True)

# 긍정키워드 포함 부정문장 제거
for intent in N_intent:
    for key in P_keyword:
        df.drop(df.loc[(df['intent'] ==intent)&df['sentence'].str.contains(key)].index, axis=0, inplace=True)         

# 필요한 데이터를 추출하기 위한 DataFrame 생성
temp_df = pd.DataFrame()
df_ = copy.deepcopy(df)

# 기존 데이터에서 긍정, 부정 문장 제거
for intent in (N_intent+P_intent):
    df_.drop(df_[df_['intent'] == intent].index, axis=0, inplace=True)

# 남은 중립문장 중 필요한만큼 문장 추출
temp_df = pd.concat([temp_df,df_.sample(n=13000)], ignore_index=True)

# 중립문장 중 긍정/부정 키워드 포함한 문장 제거
for key in (N_keyword+P_keyword):
    temp_df.drop(temp_df.loc[temp_df['sentence'].str.contains(key)].index, axis=0, inplace=True)

In [ ]:
# 필요한만큼 긍정/부정문장 추출
for intent in (N_intent+P_intent):
    if df.loc[df['intent']==intent, 'intent'].count() > 10000:
        temp = pd.concat([temp,df[df['intent'] == intent].sample(n=10000)], ignore_index=True)
    elif df.loc[df['intent']==intent, 'intent'].count() > 1000:
        temp = pd.concat([temp,df[df['intent'] == intent].sample(n=1000)], ignore_index=True)
    else :
        temp = pd.concat([temp,df[df['intent'] == intent]], ignore_index=True)

temp.loc[(temp['intent'] == '(표현) 부정감정 표현하기')|(temp['intent'] == '(언약) 거절하기')|
         (temp['intent'] == '(단언) 반박하기'), 'intent'] = '부정'
temp.loc[(temp['intent'] == '(표현) 긍정감정 표현하기')|(temp['intent'] == '(표현) 감사하기'), 'intent'] = '긍정'
temp.loc[(temp['intent'] != '부정')&(temp['intent'] != '긍정'), 'intent'] = '중립'

In [42]:
df = copy.deepcopy(temp)

In [ ]:
temp = list(df['intent'].unique())
for i in temp :
    print(i,df.loc[df['intent']==i,'intent'].count())

In [ ]:
# 클래수 개수
lable_num = len(df['intent'].unique())
lable_num

In [ ]:
label_dict = {0: '긍정', 1: '부정', 2: '중립'}

for idx, intent_lab in label_dict.items() :
    df.loc[df['intent'] == intent_lab, 'intent'] = idx

print(label_dict)
df = df.sample(frac=1).reset_index(drop=True)
df

In [ ]:
train_data = pd.DataFrame()

len_ = df['intent'].count()

train_data = pd.concat([train_data,df.loc[:(len_*8)/10]],ignore_index=True)
print(train_data)

In [ ]:
test_data = pd.DataFrame()

len_ = df['intent'].count()

test_data = pd.concat([test_data,df.loc[(len_*8)/10:]],ignore_index=True)
print(test_data)

In [ ]:
max_seq_len = max(df['sentence'].apply(lambda x: len(str(x))))

sent1 = train_data['sentence'].iloc[0]
print(max_seq_len)
print('문 장1 :',sent1)

In [ ]:
# 정수인코딩
encoded_result = tokenizer.encode(sent1,max_length=max_seq_len, pad_to_max_length=True)
print(encoded_result)
print('길 이 :', len(encoded_result))

In [ ]:
# 세그멘트인코딩
print([0]*max_seq_len)

In [ ]:
# 어텐션마스크인코딩
valid_num = len(tokenizer.encode(sent1))
print(valid_num * [1] + (max_seq_len - valid_num) * [0])

In [64]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):

    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []

    for example, label in tqdm(zip(examples, labels), total=len(examples)):
        # input_id는 워드임베딩을 위한 문장의 정수인코딩
        input_id = tokenizer.encode(example, max_length=max_seq_len, pad_to_max_length=True)

        # attention_mask는 실제단어가 위치하면 1, 패딩의 위치에는 0인 시퀀스.
        padding_count = input_id.count(tokenizer.pad_token_id)
        attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count

        # token_type_id은 세그먼트인코딩
        token_type_id = [0] * max_seq_len

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention masklength {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length{} vs {}".format(len(token_type_id), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)

    data_labels = np.asarray(data_labels, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), data_labels

In [ ]:
train_X, train_y = convert_examples_to_features(train_data['sentence'], train_data['intent'],
                                              max_seq_len=max_seq_len, tokenizer=tokenizer)

In [ ]:
test_X, test_y = convert_examples_to_features(test_data['sentence'], test_data['intent'],
                                              max_seq_len=max_seq_len, tokenizer=tokenizer)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model = TFBertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=lable_num, from_pt=True)
model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['accuracy'])

In [ ]:
model.fit(train_X, train_y, epochs=4, batch_size=128, validation_split=0.2)

In [ ]:
results = model.evaluate(test_X, test_y, batch_size=64)
print("test loss, test acc: ", results)

In [ ]:
def sentiment_predict(new_sentence):
    input_id = tokenizer.encode(new_sentence, max_length=max_seq_len, pad_to_max_length=True)

    padding_count = input_id.count(tokenizer.pad_token_id)
    attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count
    token_type_id = [0] * max_seq_len

    input_ids = np.array([input_id])
    attention_masks = np.array([attention_mask])
    token_type_ids = np.array([token_type_id])

    encoded_input = [input_ids, attention_masks, token_type_ids]

    score = np.argmax(model.predict(encoded_input)[0])
    
    print('sentence :',new_sentence)
    print(label_dict[score])

In [ ]:
MODEL_SAVE_PATH = os.path.join("NP_classifier")
print(MODEL_SAVE_PATH)

if os.path.exists(MODEL_SAVE_PATH):
    print(f"{MODEL_SAVE_PATH} -- Folder already exists \n")

else:
    os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
    print(f"{MODEL_SAVE_PATH} -- Folder create complete \n")

model.save_pretrained(MODEL_SAVE_PATH)
tokenizer.save_pretrained(MODEL_SAVE_PATH)

In [ ]:
new_sentence = input('sentence > ')
sentiment_predict(new_sentence)